# 멜론 크롤링
1.노래 리스트 받아와서 쿼리 바꿔서 검색
: 노래 리스트는 json파일로 저장 - key=노래, value:가수이름
그 안에서 id 찾아서 -> 노래 id 리스트업
2.노래 id로 노래상세페이지 쿼리바꿔가면서 데이터 크롤링 - 발매일, 장르, 좋아요수, 댓글수

In [2]:
import time
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import re
import json

In [5]:
gaon_df = pd.read_csv('./new_gaon_data.csv', index_col=0)
gaon_df = gaon_df.loc[gaon_df['year']==2016]
gaon_df = gaon_df[:int(len(gaon_df)/2)]
gaon_df

,year,week_num,start_date,end_date,rank,music,singer,album
15600,2016,1,2015.12.27,2016.01.02,1,또 하루,Gary (개리),또 하루
15601,2016,1,2015.12.27,2016.01.02,2,어땠을까,김나영,어땠을까
15602,2016,1,2015.12.27,2016.01.02,3,소녀,오혁,응답하라 1988 OST Part 3
15603,2016,1,2015.12.27,2016.01.02,4,걱정말아요 그대,이적,응답하라 1988 OST Part 2
15604,2016,1,2015.12.27,2016.01.02,5,함께,노을,응답하라 1988 OST Part 7
...,...,...,...,...,...,...,...,...
18245,2016,23,2016.05.29,2016.06.04,94,월량대표아적심,홍진영,월량대표아적심
18246,2016,22,2016.05.22,2016.05.28,97,아주 오래된 연인들,하현우,복면가왕 62회
18247,2016,20,2016.05.08,2016.05.14,100,벚꽃이 지면,아이오아이 (I.O.I),아이오아이 (I.O.I) 1st Mini Album `Chrysalis`
18248,2016,21,2016.05.15,2016.05.21,99,I`m Not The Only One,Sam Smith,In The Lonely Hour


In [6]:
gaon = pd.read_csv('./new_gaon_data.csv', index_col=0)
gaon = gaon.loc[gaon['year']==2016]
gaon = gaon[2650:]
gaon

,year,week_num,start_date,end_date,rank,music,singer,album
18250,2016,26,2016.06.19,2016.06.25,90,그의 바다,백예린,Bye bye my blue
18251,2016,24,2016.06.05,2016.06.11,93,Shut Up & Groove,헤이즈 (Heize),And July
18252,2016,25,2016.06.12,2016.06.18,92,돌아오지마,헤이즈 (Heize),돌아오지마
18253,2016,23,2016.05.29,2016.06.04,95,공감,정준영,공감
18254,2016,22,2016.05.22,2016.05.28,98,Everytime,"첸(CHEN), 펀치 (Punch)",태양의 후예 OST Part.2
...,...,...,...,...,...,...,...,...
20895,2016,53,2016.12.25,2016.12.31,96,네 생각,존 박,네 생각
20896,2016,53,2016.12.25,2016.12.31,97,Forever,비와이(BewhY),쇼미더머니 5 Episode 3
20897,2016,53,2016.12.25,2016.12.31,98,Lotto,EXO,LOTTO - The 3rd Album Repackage
20898,2016,53,2016.12.25,2016.12.31,99,Mia & Sebastian`s Theme,Justin Hurwitz,La La Land (영화 `라라랜드` OST)


In [133]:
df_2015 = pd.read_csv('melon_2016_103개.csv')
df_2015_list = df_2015['music'].unique().tolist()
len(df_2015_list)

102

In [126]:
df_2015

,music,singer,release_date,category,music_like,movie_like,movie_views
0,걱정말아요 그대,이적,2015.11.07,"발라드, 국내드라마",175218,NaN,NaN
1,소녀,오혁,2015.11.21,"발라드, 국내드라마",199121,1294.0,744347.0
2,어땠을까,김나영,2015.12.30,발라드,72672,NaN,NaN
3,매일 그대와,소진,2015.12.26,"발라드, 국내드라마",20213,548.0,164752.0
4,사랑이 맞을거야,윤미래,2015.12.14,발라드,42379,624.0,218061.0
...,...,...,...,...,...,...,...
97,잠깐만,아이오아이,2016.10.17,댄스,52164,NaN,NaN
98,내 옆에 그대인 걸,다비치,2016.10.13,발라드,61879,1036.0,250596.0
99,Begin,방탄소년단,2016.10.10,R&B/Soul,86869,NaN,NaN
100,빌려줄게,신용재,2016.10.13,발라드,78641,372.0,87717.0


2017년 데이터 왜 59개만 뽑혔을까..?

In [111]:
df_2016 = pd.read_csv('melon_2016.csv')
df_2016_list = df_2016['music'].unique().tolist()
len(df_2016_list)

148

In [117]:
df_2016_list

['에잇',
 '좋은 사람 있으면 소개시켜줘',
 'Punch',
 '살짝 설렜어',
 '나비와 고양이',
 '아로하',
 'Candy',
 '흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야',
 '화려하지 않은 고백',
 '사랑하게 될 줄 알았어',
 '시작',
 '깡 Official Remix',
 'Dolphin',
 '덤더럼(Dumhdurum',
 '처음처럼',
 '미리 메리 크리스마스',
 '샤넬',
 '벗',
 '헤어져줘서 고마워',
 '영화 속에 나오는 주인공처럼',
 '나의 어깨에 기대어요',
 'Amanda',
 '헤어지자',
 '중2병',
 '주저하는 연인들을 위해',
 'Speechless',
 '꿈속에 너',
 '숨겨진 세상',
 '잘 살고 있다',
 '허전해',
 '차가워진 이 바람엔 우리가 써있어(Bad Love',
 '노을',
 'BAND',
 '날 보러 와요',
 'LION',
 'Feel Special',
 'unlucky',
 'Snowman',
 'Santa Tell Me',
 '너에게 못했던 내 마지막 말은',
 '가을밤 떠난 너',
 '술이 문제야',
 '이별주',
 '시간의 바깥',
 '첫 겨울이니까',
 'Birthday',
 'Maniac',
 '바빠서',
 'Moon',
 'Filter',
 '욱',
 'Senorita',
 '시차',
 '너의 하루는 좀 어때',
 '둘만의 세상으로 가',
 '소우주',
 'Paris In The Rain',
 '새 사랑',
 '봄날',
 '인기',
 '내게 들려주고 싶은 말',
 '조금 취했어',
 'ALL FOR YOU',
 '워커홀릭',
 '사랑에 연습이 있었다면',
 '이 번호로 전화해줘',
 '빌었어',
 'Loveship',
 'IDOL',
 '기억해줘요 내 모든 날과 그때를',
 '마음이 말하는 행복',
 'Flower',
 '어떤 날엔',
 'Show Yourself',
 '니 소식',
 'Black Swan',
 '사계',
 'Obsession',
 '불티',
 '있어줘

In [115]:
gaon_df = pd.read_csv('new_gaon_data.csv', index_col=0)
gaon_2016 = gaon_df.loc[gaon_df['year']==2016]
gaon_2016_list = gaon_2016['music'].unique()

In [127]:
len(gaon_2016_list)

495

In [122]:
gaon_df = pd.read_csv('./new_gaon_data.csv', index_col=0)
gaon_df = gaon_df.loc[gaon_df['year']==2016]
music_list = gaon_df['music'].unique().tolist()


# for music in music_list:
    
#     singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
#     print('singer 전: ', singer)
    
#     # 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
#     if singer.find('(') != -1:
#         p = re.compile('\(([^)]+)')
#         singer_bracket = p.findall(singer)[0]
#         print('singerbracet: ', singer_bracket)
#         singer = re.sub(' \([^)]*\)','', singer) #정규표현식으로 괄호 안의 내용 모두 삭제
#         print('singer 정규표현식 후: ', singer)
#     # 괄호 없는 경우는 변수에 가수 이름 그대로 저장
#     else:
#         singer_bracket = singer
#         print('노래에 괄호 없으면: ', singer_bracket)

#     # 가수 이름의 공백을 '+'로 바꿔줌
#     singer = singer.replace(' ', '+')

#     music = music.replace(' ', '+')


#     # 오류 방지를 위해 가수 3명이상 같이 부른 노래면 맨 앞 사람만으로 검색하기
#     singer_list = singer.split('+')

#     print(singer_list, music)
#     print('------------------------------------------------------')

singer 전:  Gary (개리)
singerbracet:  개리
singer 정규표현식 후:  Gary
['Gary'] 또+하루
------------------------------------------------------
singer 전:  김나영
노래에 괄호 없으면:  김나영
['김나영'] 어땠을까
------------------------------------------------------
singer 전:  오혁
노래에 괄호 없으면:  오혁
['오혁'] 소녀
------------------------------------------------------
singer 전:  이적
노래에 괄호 없으면:  이적
['이적'] 걱정말아요+그대
------------------------------------------------------
singer 전:  노을
노래에 괄호 없으면:  노을
['노을'] 함께
------------------------------------------------------
singer 전:  싸이 (Psy)
singerbracet:  Psy
singer 정규표현식 후:  싸이
['싸이'] DADDY
------------------------------------------------------
singer 전:  윤미래 (T)
singerbracet:  T
singer 정규표현식 후:  윤미래
['윤미래'] 사랑이+맞을거야
------------------------------------------------------
singer 전:  소진 (걸스데이)
singerbracet:  걸스데이
singer 정규표현식 후:  소진
['소진'] 매일+그대와
------------------------------------------------------
singer 전:  수지, 백현 (BAEKHYUN)
singerbracet:  BAEKHYUN
singer 정규표현식 후:  수지, 백현
['수지,', '백현'] Dr

In [97]:
# 최종 df 읽어오기
gaon_df = pd.read_csv('new_gaon_data.csv', index_col=0)


# print(len(list1), gaon_df)
# len(gaon_df)

music_list = gaon_df['music'].loc[gaon_df['year']==2016].unique().tolist()
len(music_list)

495

# 괄호 삭제 전처리

In [70]:

# 최종 df 읽어오기
gaon_df = pd.read_csv('./gaon_data.csv', index_col=0)
gaon_df = gaon_df.loc[gaon_df['year']==2021]
len(gaon_df)/2

2600.0

In [15]:
gaon_df

,year,week_num,start_date,end_date,rank,music,singer,album
0,2013,11,2013.03.10,2013.03.16,1,거북이,다비치,Mystic Ballad Part 1
1,2013,11,2013.03.10,2013.03.16,2,눈꽃,거미,"그 겨울, 바람이 분다 OST Part 3"
2,2013,11,2013.03.10,2013.03.16,3,눈꽃,거미,"그 겨울, 바람이 분다 OST Part 3"
3,2013,11,2013.03.10,2013.03.16,4,있다 없으니까 (Gone Not Around Any Longer),씨스타19,있다 없으니까
4,2013,11,2013.03.10,2013.03.16,5,라면인건가,AKMU (악뮤),SBS K팝 스타 시즌2 TOP 10 Part 1
...,...,...,...,...,...,...,...,...
53695,2023,15,2023.04.09,2023.04.15,100,The Drum,Alan Walker,The Drum
53696,2023,4,2023.01.22,2023.01.28,99,Love War (Feat. BE`O),YENA(최예나),Love War
53697,2023,5,2023.01.29,2023.02.04,99,Love War (Feat. BE`O),YENA(최예나),Love War
53698,2023,4,2023.01.22,2023.01.28,100,The Drum,Alan Walker,The Drum


In [16]:
new_music_list = []
for i in range(len(gaon_df)):
    music = gaon_df['music'].iloc[i]
    if music.find('(') != -1:
        music = re.sub(' \([^)]*\)', '', music) #정규표현식으로 괄호 안의 내용 모두 삭제
    new_music_list.append(music)
gaon_df['music'] = new_music_list

In [18]:
gaon_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53700 entries, 0 to 53699
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        53700 non-null  int64 
 1   week_num    53700 non-null  int64 
 2   start_date  53700 non-null  object
 3   end_date    53700 non-null  object
 4   rank        53700 non-null  int64 
 5   music       53700 non-null  object
 6   singer      53700 non-null  object
 7   album       53700 non-null  object
dtypes: int64(3), object(5)
memory usage: 3.7+ MB


In [20]:
gaon_df.to_csv('new_gaon_data.csv')

In [23]:
new_gaon_data = pd.read_csv('new_gaon_data.csv', index_col=0)
new_gaon_data

,year,week_num,start_date,end_date,rank,music,singer,album
0,2013,11,2013.03.10,2013.03.16,1,거북이,다비치,Mystic Ballad Part 1
1,2013,11,2013.03.10,2013.03.16,2,눈꽃,거미,"그 겨울, 바람이 분다 OST Part 3"
2,2013,11,2013.03.10,2013.03.16,3,눈꽃,거미,"그 겨울, 바람이 분다 OST Part 3"
3,2013,11,2013.03.10,2013.03.16,4,있다 없으니까,씨스타19,있다 없으니까
4,2013,11,2013.03.10,2013.03.16,5,라면인건가,AKMU (악뮤),SBS K팝 스타 시즌2 TOP 10 Part 1
...,...,...,...,...,...,...,...,...
53695,2023,15,2023.04.09,2023.04.15,100,The Drum,Alan Walker,The Drum
53696,2023,4,2023.01.22,2023.01.28,99,Love War,YENA(최예나),Love War
53697,2023,5,2023.01.29,2023.02.04,99,Love War,YENA(최예나),Love War
53698,2023,4,2023.01.22,2023.01.28,100,The Drum,Alan Walker,The Drum


# 데이터 확인

In [40]:
melon_df = pd.read_csv('melon_2020.csv')
melon_df

,music,singer,release_date,category,music_like,movie_like,movie_views
0,Punch,NCT 127,2020.05.19,랩/힙합,76843,5754.0,54375.0
1,살짝 설렜어,오마이걸,2020.04.27,댄스,193045,6199.0,754205.0
2,나비와 고양이),볼빨간사춘기,2020.05.13,"인디음악, 포크/블루스",176790,2169.0,151330.0
3,아로하,조정석,2020.03.27,"발라드, 국내드라마",244547,2172.0,350980.0
4,에잇,아이유,2020.05.06,록/메탈,391397,13165.0,843993.0
...,...,...,...,...,...,...,...
250,Dance Monkey,Tones And I,2019.08.09,POP,178339,425.0,168162.0
251,"너에게 난, 나에게 넌",미도와 파라솔,2020.05.29,"발라드, 국내드라마",78330,2718.0,194306.0
252,내 눈물 모아,휘인,2020.05.01,"발라드, 국내드라마",62030,1275.0,70521.0
253,Blueming,아이유,2019.11.18,록/메탈,353455,11149.0,958040.0


In [39]:
gaon_df = pd.read_csv('./gaon_data.csv', index_col=0)
gaon_df = gaon_df.loc[gaon_df['year']==2020]
len(gaon_df['music'].unique().tolist())

264

In [46]:
# 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
test='나비와 고양이)'
if test.find(')') != -1:
    test = test.replace(')','')
    print(test)

나비와 고양이


# 2023년 수집 데이터 확인

In [24]:
gaon_df = pd.read_csv('./new_gaon_data.csv', index_col=0)
gaon_df = gaon_df.loc[gaon_df['year']==2023]

In [25]:
music_list = gaon_df['music'].unique().tolist()
print(len(music_list), len(gaon_df))
# music_list

117 1500


In [26]:
melon_df = pd.read_csv('./melon.csv')
melon_music = melon_df['music'].unique().tolist()
len(melon_music)

114

In [28]:
for m in music_list:
    if m not in melon_music:
        print(m)

사랑인가 봐
I AM
해요


In [34]:
gaon_df = pd.read_csv('./new_gaon_data.csv', index_col=0)
gaon_df = gaon_df.loc[gaon_df['year']==2022]
gaon_df.head(50)

,year,week_num,start_date,end_date,rank,music,singer,album
46900,2022,17,2022.04.17,2022.04.23,1,우리들의 블루스,임영웅,IM HERO
46901,2022,17,2022.04.17,2022.04.23,2,봄여름가을겨울,BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
46902,2022,17,2022.04.17,2022.04.23,3,LOVE DIVE,IVE (아이브),LOVE DIVE
46903,2022,17,2022.04.17,2022.04.23,4,TOMBOY,(여자)아이들,I NEVER DIE
46904,2022,17,2022.04.17,2022.04.23,5,Feel My Rhythm,레드벨벳(Red Velvet),`The ReVe Festival 2022 - Feel My Rhythm`
46905,2022,17,2022.04.17,2022.04.23,6,사랑인가 봐,멜로망스(Melomance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)
46906,2022,17,2022.04.17,2022.04.23,7,GANADARA,박재범,GANADARA
46907,2022,18,2022.04.24,2022.04.30,1,봄여름가을겨울,BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
46908,2022,17,2022.04.17,2022.04.23,8,취중고백,김민석,취중고백
46909,2022,18,2022.04.24,2022.04.30,2,LOVE DIVE,IVE (아이브),LOVE DIVE


# 문제1 : 이번엔 뮤비가 없는 것들로만 데이터가 뽑힘 
# 수정사항 : yield를 두번하는걸 한번으로 합쳐서 movie로 거쳐서 데이터 보내기
# 결과 : 그래도 뮤비 없는 걸로만 데이터 뽑힘

In [37]:
# test = pd.read_csv('./melon.csv')
# test_list = test['music'].unique().tolist()

In [48]:
# test_2 = pd.read_csv('./melon_test2.csv')
# test_list_2 = test_2['music'].unique().tolist()

In [40]:
intersection = set(test_list) & set(music_list)
print(len(list(intersection)))

41


In [41]:
in_list = []
no_list = []
for t in test_list:
    if t in music_list:
        in_list.append(t)
    else:
        no_list.append(t)

In [47]:
in_list

['크리스마스니까',
 '매력있어',
 '카페인',
 '되돌리다',
 '사랑은 이렇게',
 'Dancing Queen',
 '안아보자',
 '싫다',
 '인어공주',
 'Yesterday',
 '이사람',
 'I Got A Boy',
 'Goodbye To Romance',
 '노가르시아',
 'One Kiss',
 '엄마를 닮았네',
 '라면인건가',
 'Dream Girl',
 '러브 레시피',
 'One Shot',
 '안녕 나야',
 '먹지',
 '행복하니',
 '저녁 하늘',
 'Sweet Dream',
 '얼음꽃',
 '기억을 걷다보면',
 '모노드라마',
 '너였으면 좋겠어',
 '24/7',
 '지우개',
 '하지 못한 말',
 '혼자라고 생각말기',
 '빗소리',
 'Mine',
 'I’m Sorry',
 'Officially Missing You, Too',
 '메뚜기월드',
 '강북멋쟁이',
 'Express 999',
 '일년전에']

In [46]:
no_list

['그런 너',
 '너 땜에 못살아',
 '섹시 보이',
 '나도 여자인데',
 '있다 없으니까',
 '눈물',
 'Special Girl',
 '니가 없을 때',
 '눈물샤워',
 '시작이 좋아',
 '그 사람']

In [7]:
# 뮤비가 없으면 모든 데이터가 저장이 되지 않는 오류 해결

gaon_df = pd.read_csv('./gaon_data_sp.csv')
gaon_df = gaon_df[:10]
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"
id_url = 'search/total/index.htm?q='


for music in music_list:
    
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    
    # 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
    if singer.find('(') != -1:
        p = re.compile('\(([^)]+)')
        singer_bracket = p.findall(singer)[0]
        singer = re.sub(' \([^)]*\)','', singer) #정규표현식으로 괄호 안의 내용 모두 삭제
    else:
        singer_bracket = singer

    # 가수 이름의 공백을 '+'로 바꿔줌
    singer = singer.replace(' ', '+')

    music = re.sub(' \([^)]*\)', '', music) #정규표현식으로 괄호 안의 내용 모두 삭제
    music = music.replace(' ', '+')

    # 오류 방지를 위해 가수 3명이상 같이 부른 노래면 맨 앞 사람만으로 검색하기
    singer_list = singer.split('+')

    print(f'음악: {music}, 가수:{singer}, 가수괄호:{singer_bracket}, singer_list:{singer_list}')

    try:
        try:
            for singer in singer_list:
                URL = base_url + id_url + singer + '+' + music
                print(URL)
                header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
                response = requests.get(URL,headers=header)
                soup = BeautifulSoup(response.text, 'html.parser')

                links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
                music_id = links.attrs['value']
                print('music_id: ', music_id)

                time.sleep(2)

                # 노래 데이터 추출
                URL_1 = base_url + f'song/detail.htm?songId={music_id}'
                response_1 = requests.get(URL_1,headers=header)
                soup = BeautifulSoup(response_1.text, 'html.parser')

                # 발매일, 장르
                release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
                category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
                
                time.sleep(2)

                # 좋아요 가져오기위해 json으로 접근
                URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={music_id}'
                response_2 = requests.get(URL_2,headers=header).text

                like_json = json.loads(response_2)
                music_like = like_json['contsLike'][0]['SUMMCNT']


                # 뮤비 고유 code 가져오기 - 상세페이지에 뮤비 없는 경우 pass
                try:
                    movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
                    movie = movie.attrs['href']
                    movie = re.sub('[a-zA-z:.(\')]', '', movie)
                    movie_id = movie.split(', ')[1]
                    time.sleep(2)

                    # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
                    URL_3 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={movie_id}'
                    response_3 = requests.get(URL_3, headers=header).text
                    movie_json = json.loads(response_3)
                    movie_like = movie_json['contsLike'][0]['LOVECNT']
                    movie_views = movie_json['contsLike'][0]['READCNT']
                    
                    print(f'뮤비 있음 노래제목:{music} 뮤비좋아요:{movie_like} 뮤비뷰수:{movie_views}')
    
                except:
                    print(f'뮤비 없음 노래제목:{music}')

                break
 
        # 가수 이름으로 검색해서 없으면 가수이름 괄호 안의 이름으로 검색 singer_bracket
        except:
            URL = base_url + id_url + singer_bracket + '+' + music
            print(URL)
            header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
            response = requests.get(URL,headers=header)
            soup = BeautifulSoup(response.text, 'html.parser')

            time.sleep(2)

            links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
            music_id = links.attrs['value']
            print('music_id: ', music_id)
            
            # 노래 데이터 추출
            URL_1 = base_url + f'song/detail.htm?songId={music_id}'
            response_1 = requests.get(URL_1,headers=header)
            soup = BeautifulSoup(response_1.text, 'html.parser')

            # 발매일, 장르
            release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
            category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
            
            time.sleep(2)

            # 좋아요 가져오기위해 json으로 접근
            URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={music_id}'
            response_2 = requests.get(URL_2,headers=header).text

            like_json = json.loads(response_2)
            music_like = like_json['contsLike'][0]['SUMMCNT']


            # 뮤비 고유 code 가져오기 - 상세페이지에 뮤비 없는 경우 pass
            try:
                time.sleep(2)
                movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
                movie = movie.attrs['href']
                movie = re.sub('[a-zA-z:.(\')]', '', movie)
                movie_id = movie.split(', ')[1]

                time.sleep(2)
                # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
                URL_3 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={movie_id}'
                response_3 = requests.get(URL_3, headers=header).text
                movie_json = json.loads(response_3)
                movie_like = movie_json['contsLike'][0]['LOVECNT']
                movie_views = movie_json['contsLike'][0]['READCNT']

                print(f'뮤비 있음 노래제목:{music} 뮤비좋아요:{movie_like} 뮤비뷰수:{movie_views}')
            except:
                print(f'뮤비 없음 노래제목:{music}')
                
    except:
        print('오류 발생 : 해당 노래 검색값 없음')
        pass

음악: Dancing+Queen, 가수:소녀시대, 가수괄호:소녀시대, singer_list:['소녀시대']
https://www.melon.com/search/total/index.htm?q=소녀시대+Dancing+Queen
music_id:  3994911
뮤비 있음 노래제목:Dancing+Queen 뮤비좋아요:1495 뮤비뷰수:627890
음악: 사랑은+이렇게, 가수:케이윌, 가수괄호:K.Will, singer_list:['케이윌']
https://www.melon.com/search/total/index.htm?q=케이윌+사랑은+이렇게
music_id:  3995922
뮤비 있음 노래제목:사랑은+이렇게 뮤비좋아요:381 뮤비뷰수:48870
음악: 되돌리다, 가수:이승기, 가수괄호:이승기, singer_list:['이승기']
https://www.melon.com/search/total/index.htm?q=이승기+되돌리다
music_id:  3968836
뮤비 있음 노래제목:되돌리다 뮤비좋아요:963 뮤비뷰수:140511
음악: 카페인, 가수:양요섭, 가수괄호:양요섭, singer_list:['양요섭']
https://www.melon.com/search/total/index.htm?q=양요섭+카페인
music_id:  3971021
뮤비 있음 노래제목:카페인 뮤비좋아요:3875 뮤비뷰수:228636
음악: 매력있어, 가수:AKMU, 가수괄호:악뮤, singer_list:['AKMU']
https://www.melon.com/search/total/index.htm?q=AKMU+매력있어
music_id:  3987723
뮤비 없음 노래제목:매력있어
음악: 크리스마스니까, 가수:성시경,+박효신,+이석훈,+서인국,+빅스, 가수괄호:VIXX, singer_list:['성시경,', '박효신,', '이석훈,', '서인국,', '빅스']
https://www.melon.com/search/total/index.htm?q=성시경,+크리스마스니까
music_id:  3

In [ ]:
# 2013년 1주차 장현승,정은지,김남주 노래 일년전에 오류 뜨는 이유
# 오류 : 검색해도 노래가 아니고 mv만 떠서 값 못 찾아서 오류
# 오류 이유 : 장현승이 최근에 이름을 able로 바꿔서 able으로 검색해야 노래가 나와서......


In [11]:
# 0. 최종df에서 노래랑 제목 가져와서 검색
gaon_df = pd.read_csv('./gaon_data_sp.csv')
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"

for music in music_list:
    
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    
    # 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
    if singer.find('(') != -1:
        p = re.compile('\(([^)]+)')
        singer_bracket = p.findall(singer)[0]
        singer = re.sub(' \([^)]*\)','', singer) #정규표현식으로 괄호 안의 내용 모두 삭제
    else:
        singer_bracket = singer

    # 가수 이름의 공백을 '+'로 바꿔줌
    singer = singer.replace(' ', '+')

    music = re.sub(' \([^)]*\)', '', music) #정규표현식으로 괄호 안의 내용 모두 삭제
    music = music.replace(' ', '+')

    # 오류 방지를 위해 가수 3명이상 같이 부른 노래면 맨 앞 사람만으로 검색하기
    singer_list = singer.split('+')

    # if len(singer_list) > 2:
    #     singer_search = singer_list[0]
    # else:
    #     singer_search = singer

    id_url = 'search/total/index.htm?q='
    time.sleep(3)
    # singer을 바꿔서 검색
    # 가수가 3명 이상이면 singer_list로 가수명 쪼개고 그 안에서 for문으로 오류 안나고 값 찾을때까지 반복
    # 만약 위에 거 안된다면 괄호안의 내용으로 검색 
    try:
        try:
            for singer in singer_list:
                URL = base_url + id_url + singer + '+' + music
                header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
                response = requests.get(URL,headers=header)
                soup = BeautifulSoup(response.text, 'html.parser')

                links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
                music_id = links.attrs['value']
                print(f'singef: {singer}, music_id: {music_id}')
                break
        except:
            URL = base_url + id_url + singer_bracket + '+' + music
            header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
            response = requests.get(URL,headers=header)
            soup = BeautifulSoup(response.text, 'html.parser')

            links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
            music_id = links.attrs['value']
            print(f'singef: {singer}, music_id: {music_id}')
    except:
        print('오류 발생 : 값 없음')
        pass
    time.sleep(3)


오류 발생 : 값 없음
오류 발생 : 값 없음


KeyboardInterrupt: 

In [ ]:
# 처리해야하는거
# singer_list 가수 여러명이면 콤마로 이어져있어서
# 다 빼주는게 좋을 거 같긴 함
# ex) 장현승, 정은지, 김남주
# singer_list[0] 이 '장현승,'

In [109]:
# 백업
# 0. 최종df에서 노래랑 제목 가져와서 검색
gaon_df = pd.read_csv('./gaon_data_sp.csv')
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"

for music in music_list:
    
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    
    # 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
    if singer.find('(') != -1:
        p = re.compile('\(([^)]+)')
        singer_bracket = p.findall(singer)[0]
        singer = re.sub(' \([^)]*\)','', singer) #정규표현식으로 괄호 안의 내용 모두 삭제
    else:
        singer_bracket = singer

    # 가수 이름의 공백을 '+'로 바꿔줌
    singer = singer.replace(' ', '+')

    music = re.sub(' \([^)]*\)', '', music) #정규표현식으로 괄호 안의 내용 모두 삭제
    music = music.replace(' ', '+')

    # 오류 방지를 위해 가수 3명이상 같이 부른 노래면 맨 앞 사람만으로 검색하기
    singer_list = singer.split('+')
    print('singer_list', singer_list)
    if len(singer_list) > 2:
        singer_search = singer_list[0]
        print('가수 3명이상', singer_search)
    else:
        singer_search = singer
        print('가수', singer)

    id_url = 'search/total/index.htm?q='

    # 검색 결과가 없는 경우, 가수 이름 괄호 안의 내용으로 검색
    try:
        URL = base_url + id_url + singer_search + '+' + music
        print(URL)
        header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
        response = requests.get(URL,headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
        music_id = links.attrs['value']
    except:

        URL = base_url + id_url + singer_bracket + '+' + music
        print(URL)
        header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
        response = requests.get(URL,headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
        music_id = links.attrs['value']

singer_list ['소녀시대']
가수 소녀시대
https://www.melon.com/search/total/index.htm?q=소녀시대+Dancing+Queen
singer_list ['케이윌']
가수 케이윌
https://www.melon.com/search/total/index.htm?q=케이윌+사랑은+이렇게
singer_list ['이승기']
가수 이승기
https://www.melon.com/search/total/index.htm?q=이승기+되돌리다
singer_list ['양요섭']
가수 양요섭
https://www.melon.com/search/total/index.htm?q=양요섭+카페인
singer_list ['AKMU']
가수 AKMU
https://www.melon.com/search/total/index.htm?q=AKMU+매력있어
singer_list ['성시경,', '박효신,', '이석훈,', '서인국,', '빅스']
가수 3명이상 성시경,
https://www.melon.com/search/total/index.htm?q=성시경,+크리스마스니까
singer_list ['써니힐']
가수 써니힐
https://www.melon.com/search/total/index.htm?q=써니힐+Goodbye+To+Romance
singer_list ['소녀시대']
가수 소녀시대
https://www.melon.com/search/total/index.htm?q=소녀시대+I+Got+A+Boy
singer_list ['효린,', '전효성,', '현아,', '니콜,', '나나']
가수 3명이상 효린,
https://www.melon.com/search/total/index.htm?q=효린,+이사람
singer_list ['정진운,', '엘조,', '이준,', '호야,', '이기광']
가수 3명이상 정진운,
https://www.melon.com/search/total/index.htm?q=정진운,+Yesterday
singer_list ['강

AttributeError: 'NoneType' object has no attribute 'attrs'

In [54]:
gaon_df['singer'].tolist()

['소녀시대',
 '케이윌 (K.Will)',
 '이승기',
 '양요섭',
 'AKMU (악뮤)',
 '성시경, 박효신, 이석훈, 서인국, 빅스 (VIXX)',
 '써니힐 (Sunny Hill)',
 '소녀시대',
 '양요섭',
 '효린, 전효성, 현아, 니콜, 나나',
 '정진운 (2am), 엘조 (틴탑), 이준 (엠블랙), 호야 (인피니트), 이기광 (비스트)',
 '소녀시대',
 '강지영, 보라, 한선화, 허가윤, 리지',
 '백지영',
 'AKMU (악뮤)',
 '제아',
 '더 원 (The One)',
 '이승기',
 '장현승, 정은지, 김남주',
 '써니힐 (Sunny Hill)',
 '소녀시대',
 '정형돈',
 '길 (리쌍)',
 '김재중 (JYJ)',
 '소녀시대',
 '써니힐 (Sunny Hill)',
 '제아',
 '백지영',
 '효린, 전효성, 현아, 니콜, 나나',
 '노홍철',
 '유재석',
 '이승기',
 '장현승, 정은지, 김남주',
 '버벌진트 (Verbla Jint)',
 '양요섭',
 'AKMU (악뮤)',
 '배치기',
 '백지영',
 '이승기',
 '긱스 (Geeks), 소유',
 '씨엔블루 (CNBLUE)',
 '버벌진트 (Verbla Jint)',
 '양요섭',
 'AKMU (악뮤)',
 '정형돈',
 '김재중 (JYJ)',
 'B.A.P',
 '인피니트H',
 '소녀시대',
 '김보경',
 '소녀시대',
 '제아',
 '인피니트H',
 '써니힐 (Sunny Hill)',
 '효린, 전효성, 현아, 니콜, 나나',
 '노을',
 '리쌍',
 '배치기',
 '알리 (ALi)',
 '포미닛 투윤 (2Yoon)',
 '이정',
 '허각, 유승우',
 '씨엔블루 (CNBLUE)',
 '정형돈',
 '써니힐 (Sunny Hill)',
 '디셈버 (December)',
 '씨스타19',
 '에일리 (Ailee)',
 '백지영',
 '씨스타19',
 '긱스 (Geeks), 소유',
 '배치기',
 '리쌍',
 '김보경',
 'MFB

In [4]:
# 1. 노래 리스트에서 가져와서 노래별로 id 찾기
# music_list = ['and july', 'cupid', '꽃']
# singer_list = ['heize', '피프티피프티', '지수']

# 띄어쓰기는 +로 바꿔주기
music_list = [m.replace(' ', '+') for m in music_list]
singer_list = [s.replace(' ', '+') for s in singer_list]

In [52]:
singer_list

NameError: name 'singer_list' is not defined

In [45]:
music_list

['Dancing+Queen',
 '사랑은+이렇게',
 '되돌리다',
 '카페인',
 '매력있어',
 '크리스마스니까',
 'Goodbye+To+Romance',
 'I+Got+A+Boy',
 '이사람',
 'Yesterday',
 '인어공주',
 '싫다',
 '안아보자',
 '그+사람+(원곡가수+이승철)',
 '일년전에',
 'Express+999',
 '강북멋쟁이',
 '엄마를+닮았네',
 'One+Kiss',
 '노가르시아',
 '메뚜기월드',
 '시작이+좋아+(Feat.+강민희)',
 '눈물샤워+(Feat.+에일리)',
 'Officially+Missing+You,+Too',
 'I’m+Sorry',
 'Mine',
 '빗소리',
 '니가+없을+때+(Feat.+Zion.T)',
 '혼자라고+생각말기',
 'Special+Girl+(Feat.+Bumkey)',
 '하지+못한+말',
 '눈물+(Feat.+유진)',
 '지우개',
 '24/7',
 '너였으면+좋겠어',
 '모노드라마',
 '기억을+걷다보면',
 '있다+없으니까+(Gone+Not+Around+Any+Longer)',
 '얼음꽃',
 '나도+여자인데+(A+Girl+In+Love)',
 'Sweet+Dream',
 '저녁+하늘',
 '이별재회',
 '행복하니',
 '섹시+보이+(Feat.+영지)',
 '너+땜에+못살아+(Feat.+버벌진트)',
 '그런+너+(Disturbance)',
 '먹지',
 'Call+Me+Maybe',
 '안녕+나야',
 'One+Shot',
 '러브+레시피',
 'Dream+Girl',
 '라면인건가',
 '어이없네',
 '어떤가요',
 '아름다워+(Beautiful)',
 '1440',
 '돌스+(Dolls)',
 '겨울사랑',
 '눈꽃',
 '이게+사랑이+아니면+(Feat.+에일리)',
 '모르시나요',
 '선인장+(Duet)',
 '긴+생머리+그녀+(Miss+Right)',
 'Black+Paradise',
 '귀요미송',
 '될+대로+되라고+해+(느낌+So+Go

In [46]:

# 2. 노래 id 찾아서 code_list 만들기
# https://www.melon.com/search/total/index.htm?q=heize+and+july

code_list=[]
for idx, music in enumerate(music_list):
    base_url = 'https://www.melon.com/search/total/index.htm?q='
    URL = base_url + singer_list[idx] + '+' + music
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    response = requests.get(URL,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
    code = links.attrs['value']
    code_list.append(code)
code_list

NameError: name 'singer_list' is not defined

In [51]:
music = music_list[1]
singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
singer = singer.replace(' ', '+')
singer

'케이윌+(K.Will)'

In [48]:
gaon_df = pd.read_csv('./gaon_data_sp.csv')
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"

for music in music_list:
    # singer찾아서 meta로 담아서 보내기
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    singer = singer.replace(' ', '+')
    music = music.replace(' ', '+')
    
    id_url = 'search/total/index.htm?q='

    URL = base_url + id_url + singer + '+' + music
    print(URL)
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    response = requests.get(URL,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
    music_id = links.attrs['value']
    print(music_id)

https://www.melon.com/search/total/index.htm?q=소녀시대+Dancing+Queen
3994911
https://www.melon.com/search/total/index.htm?q=케이윌+(K.Will)+사랑은+이렇게


AttributeError: 'NoneType' object has no attribute 'attrs'

In [ ]:
# 노래 제목, 가수 이름으로 url에 넣어서 노래의 고유 아이디 구하기
# 노래 id로 발매일, 장르, 좋아요수, 댓글수, 뮤비고유코드 얻기
    # 뮤비 고유 코드로는 뮤비 좋아요수와 뷰 수 얻기


In [145]:
# 노래 id로 노래상세페이지 쿼리바꿔가면서 데이터 크롤링 
# - 발매일, 장르, 좋아요수, 댓글수, 뮤비고유코드 ( -> 뮤비 좋아요, 뷰)

import json
import re

for code in code_list:
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    URL = f'https://www.melon.com/song/detail.htm?songId={code}'
    response_1 = requests.get(URL,headers=header)
    soup = BeautifulSoup(response_1.text, 'html.parser')

    # 발매일, 장르
    release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
    category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
    
    
    # 좋아요 가져오기위해 json으로 접근
    URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={code}'
    response_2 = requests.get(URL_2,headers=header).text

    like_json = json.loads(response_2)
    like_count = like_json['contsLike'][0]['SUMMCNT']

    print(f'노래코드:{code} 발매일{release_date}, 장르{category} 좋아요{like_count}')
    print('--------------------------------------------------------------')


    # 뮤비 고유 code 가져오기
    movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
    movie = movie.attrs['href']
    movie = re.sub('[a-zA-z:.(\')]', '', movie)
    movie_code = movie.split(', ')[1]

    # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
    URL_3 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={movie_code}'
    response_3 = requests.get(URL_3, headers=header).text
    movie_json = json.loads(response_3)
    movie_like = movie_json['contsLike'][0]['LOVECNT']
    movie_views = movie_json['contsLike'][0]['READCNT']
    print(f'뮤비 코드:{movie_code} 뮤비좋아요:{movie_like}, 뮤비뷰수{movie_views}')


뮤비 코드:50152059 뮤비좋아요:2309, 뮤비뷰수588781
노래코드:8271355 발매일2016.07.18, 장르랩/힙합 좋아요182074
--------------------------------------------------------------
뮤비 코드:50260014 뮤비좋아요:217, 뮤비뷰수8829
노래코드:36206259 발매일2023.02.24, 장르댄스 좋아요50755
--------------------------------------------------------------
뮤비 코드:50260334 뮤비좋아요:1488, 뮤비뷰수116284
노래코드:36331121 발매일2023.03.31, 장르댄스 좋아요86179
--------------------------------------------------------------
